# Putting it all together

In [1]:
# !pip install trulens_eval

In [66]:
# #imports
# from openai import OpenAI
# from glob import glob
# from llama_index import download_loader, VectorStoreIndex, ServiceContext
# from llama_index.vector_stores import MilvusVectorStore
# from llama_index.embeddings import HuggingFaceEmbedding
# from llama_index import ServiceContext
# from llama_index.llms import PaLM
# from llama_index.embeddings import GooglePaLMEmbedding
# from llama_index.memory import ChatMemoryBuffer
# import numpy as np
# from trulens_eval import TruLlama, Tru, Query, Feedback, feedback
# import google.generativeai as palm
# import os
# import subprocess
# import sys

In [1]:
# !pip install google-cloud-aiplatform


In [3]:
!git clone https://github.com/being-invincible/hydroponics-grow-guide-dataset

Cloning into 'hydroponics-grow-guide-dataset'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 64 (delta 21), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (64/64), 35.54 KiB | 1.54 MiB/s, done.
Resolving deltas: 100% (21/21), done.


## Parameters

In [1]:
from os import environ

# 1. Set up the name of the collection to be created.
COLLECTION_NAME = 'hydroponics_knowledge_base'

# 2. Set up the dimension of the embeddings.
DIMENSION = 1536

# 3. Set the inference parameters
BATCH_SIZE = 128
TOP_K = 3

# 4. Set up the connection parameters for your Zilliz Cloud cluster.
URI = environ['CLUSTER_ENDPOINT']

TOKEN = environ['API_TOKEN']

# Palm API
palm_api_key = environ['PALM_API_KEY']

# OpenAI API key
environ["OPENAI_API_KEY"] = environ['OPEN_AI_KEY']
# environ["TOKENIZERS_PARALLELISM"] = "false"

In [40]:
# from llama_index.vector_stores import MilvusVectorStore
# from llama_index import StorageContext, VectorStoreIndex


# vector_store = MilvusVectorStore(
#     uri=URI, 
#     token=TOKEN, 
#     collection_name=COLLECTION_NAME, 
#     similarity_metric="L2",
#     dim=DIMENSION,
# )

# storage_context = StorageContext.from_defaults(vector_store=vector_store)

## Document Loader

In [4]:
from llama_index import download_loader
from glob import glob

# Load the markdown reader from the hub
MarkdownReader = download_loader("MarkdownReader")
markdownreader = MarkdownReader()

# Grab all markdown files and convert them using the reader
docs = []
for file in glob("./hydroponics-grow-guide-dataset/*.md", recursive=True):
    docs.extend(markdownreader.load_data(file=file))
print(len(docs))

103


## PaLM Vertex AI

In [5]:
import google.generativeai as palm

palm.configure(api_key=palm_api_key)

In [6]:
models = [
    m
    for m in palm.list_models()
    if "generateText" in m.supported_generation_methods
]
model = models[0].name
print(model)

models/text-bison-001


In [7]:
from llama_index import ServiceContext
from llama_index.llms import PaLM
# PaLM(api_key=palm_api_key
llm=PaLM(api_key=palm_api_key)

## Testing with open source embedding

In [9]:
# from llama_index import download_loader, VectorStoreIndex, ServiceContext
# from llama_index.vector_stores import MilvusVectorStore
# from llama_index.embeddings import HuggingFaceEmbedding

# # Push all markdown files into Zilliz Cloud
# vector_store = MilvusVectorStore(
#     uri=URI, 
#     token=TOKEN, 
#     collection_name=COLLECTION_NAME, 
#     similarity_metric="L2",
#     dim=DIMENSION,
# )

# embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L12-v2")
# service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

# index = VectorStoreIndex.from_documents(
#     documents=docs, 
#     service_context=service_context,
#     show_progress=True,
#     llm=llm
# )

# print(index.vector_store)

In [10]:
from llama_index import download_loader, VectorStoreIndex, ServiceContext
from llama_index.vector_stores import MilvusVectorStore

In [11]:
# print(service_context)
# print(index.service_context)

## Using PaLM Embedding & PaLM Text Model

In [8]:
from llama_index import ServiceContext
from llama_index.llms import PaLM
from llama_index.embeddings import GooglePaLMEmbedding

# PaLM(api_key=palm_api_key
llm=PaLM(api_key=palm_api_key)
embed_model = GooglePaLMEmbedding("models/embedding-gecko-001", api_key=palm_api_key)
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

In [12]:
index = VectorStoreIndex.from_documents(
    documents=docs, 
    service_context=service_context,
    show_progress=True,
    llm=llm
)

Parsing nodes:   0%|          | 0/103 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/103 [00:00<?, ?it/s]

## Loading from Vector DB

In [51]:
# from llama_index.vector_stores import MilvusVectorStore
# vector_store = MilvusVectorStore(
#     uri=URI, 
#     token=TOKEN, 
#     collection_name=COLLECTION_NAME, 
#     similarity_metric="L2",
#     dim=DIMENSION,
# )


In [52]:
# from llama_index import VectorStoreIndex
# index = VectorStoreIndex.from_documents(
#     vector_store=vector_store,
#     service_context=service_context,
#     documents=


# )

## Query Engine

In [13]:
# # Chat Engine
# query_engine = index.as_chat_engine(chat_mode="context",
#      verbose=True)
# response = query_engine.chat("What is greens?")
# print(str(response))

In [14]:
query_engine = index.as_query_engine()
response = query_engine.query("What is hydroponics?")
print(str(response))

growing plants in water instead of soil


## Trulens

In [15]:
from trulens_eval import TruLlama, Tru

l = TruLlama(query_engine)

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [16]:
response = l.query("How long it will take for kale to harvest?")
print(response)

Function <method-wrapper '__call__' of method object at 0x164088cc0> has not been instrumented. This may be ok if it will call a function that has been instrumented exactly once. Otherwise unexpected results may follow. You can use `AddInstruments.method` of `trulens_eval.instruments` before you use the `TruLlama` wrapper to make sure `__call__` does get instrumented. `TruLlama` method `print_instrumented` may be used to see methods that have been instrumented. 



`query` will be deprecated soon; To record results of your app's execution, use one of these options to invoke your app:
    (1) Use the `with_` method:
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        result = tru_app_recorder.with_(app.query, ...args/kwargs-to-app.query...)
        ```
    (2) Use TruLlama as a context manager: 
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        with tru_app_recorder:
            result = app.query(...args/kwargs-to-app.query...)
        
        ```


`query_with_record` will be deprecated soon; To record results of your app's execution, use one of these options to invoke your app:
    (1) Use the `with_record` method:
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        result, record = tru_app_recorder.with_record(app.query, ...args/kwargs-to-app.query...)
        ```
    (2) Use TruLlam

In [17]:
tru = Tru()
tru.run_dashboard()

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.0.4:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [19]:
l.query("What is hydroponics?")

Function <method-wrapper '__call__' of method object at 0x1651a6140> has not been instrumented. This may be ok if it will call a function that has been instrumented exactly once. Otherwise unexpected results may follow. You can use `AddInstruments.method` of `trulens_eval.instruments` before you use the `TruLlama` wrapper to make sure `__call__` does get instrumented. `TruLlama` method `print_instrumented` may be used to see methods that have been instrumented. 



`query` will be deprecated soon; To record results of your app's execution, use one of these options to invoke your app:
    (1) Use the `with_` method:
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        result = tru_app_recorder.with_(app.query, ...args/kwargs-to-app.query...)
        ```
    (2) Use TruLlama as a context manager: 
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        with tru_app_recorder:
            result = app.query(...args/kwargs-to-app.query...)
        
        ```


`query_with_record` will be deprecated soon; To record results of your app's execution, use one of these options to invoke your app:
    (1) Use the `with_record` method:
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        result, record = tru_app_recorder.with_record(app.query, ...args/kwargs-to-app.query...)
        ```
    (2) Use TruLlam

Response(response='a method of growing plants in water instead of soil.', source_nodes=[NodeWithScore(node=TextNode(id_='968f6a70-5b27-4734-83a2-331712e59aa4', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='38049149-6557-48d2-9fdb-a9eba2b72ecc', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='1893e85335735985297311aad9239f9e05482b9f7b477e0ce41a854edfe16771'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='65ae1878-7eca-4845-a2ba-4237562747df', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='7d69d9169c2490728d8ce5a724194f78d8673942a2b9f739d1d3191cc1cdde40'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='6445ddc2-0488-44ca-a520-17bde3c9a56f', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='31ed6c6e329e7d4dff353ec497c7ad0161eb436f9b95acbfd082bd8c413c1d24')}, hash='82ec6203b3414721fb569fd2582087b4064b6b0d3655e4ad0979d113ab8dd603', tex

In [20]:
l.query("What is aquaponics?")

Function <method-wrapper '__call__' of method object at 0x16597a780> has not been instrumented. This may be ok if it will call a function that has been instrumented exactly once. Otherwise unexpected results may follow. You can use `AddInstruments.method` of `trulens_eval.instruments` before you use the `TruLlama` wrapper to make sure `__call__` does get instrumented. `TruLlama` method `print_instrumented` may be used to see methods that have been instrumented. 



`query` will be deprecated soon; To record results of your app's execution, use one of these options to invoke your app:
    (1) Use the `with_` method:
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        result = tru_app_recorder.with_(app.query, ...args/kwargs-to-app.query...)
        ```
    (2) Use TruLlama as a context manager: 
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        with tru_app_recorder:
            result = app.query(...args/kwargs-to-app.query...)
        
        ```


`query_with_record` will be deprecated soon; To record results of your app's execution, use one of these options to invoke your app:
    (1) Use the `with_record` method:
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        result, record = tru_app_recorder.with_record(app.query, ...args/kwargs-to-app.query...)
        ```
    (2) Use TruLlam

Response(response='Aquaponics is a method of combining aquaculture (raising fish) with hydroponics (growing plants without soil). This can be done in a variety of ways, but the most common system is to use a tank of fish to provide nutrients for the plants, and the plants to filter the water for the fish.\n\nAquaponics is a sustainable method of food production that can be used to grow a variety of different crops, including fruits, vegetables, and herbs. It is also a more efficient use of water than traditional farming methods, and it can help to reduce the amount of pollution produced.\n\nOne of the main benefits of aquaponics is that it can be used in a variety of different settings, including urban areas and greenhouses. This makes it a viable option for people who do not have access to traditional farming land.\n\nAquaponics is also a relatively easy method of food production to get started with, and it can be a fun and rewarding hobby.', source_nodes=[NodeWithScore(node=TextNode(

## Trulens Compilation

In [21]:
from trulens_eval import Feedback, Tru, TruLlama
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI as OAI

In [22]:
import numpy as np

# Initialize provider class
openai = OAI()

grounded = Groundedness(groundedness_provider=OAI())

# Define a groundedness feedback function
f_groundedness = Feedback(grounded.groundedness_measure_with_cot_reasons).on(
    TruLlama.select_source_nodes().node.text.collect()
    ).on_output(
    ).aggregate(grounded.grounded_statements_aggregator)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on_input_output()

# Question/statement relevance between question and each context chunk.
f_qs_relevance = Feedback(openai.qs_relevance).on_input().on(
    TruLlama.select_source_nodes().node.text
    ).aggregate(np.mean)

tru_query_engine_recorder = TruLlama(query_engine,
    app_id='SHAi_App',
    feedbacks=[f_groundedness, f_qa_relevance, f_qs_relevance])

tru.run_dashboard()

✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.app.query.rets.source_nodes[:].node.text.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In qs_relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In qs_relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .
Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.0.4:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [ ]:
with tru_query_engine_recorder as recording:
    query_engine.query("How long will lettuce take to grow?")